## Dependencias

In [1]:
import pandas as pd
import json #JSON = Java Script Object Notation
from sqlalchemy import create_engine
from sqlalchemy.types import VARCHAR,FLOAT,INTEGER,DATE,CHAR,DATETIME
import os

pd.set_option('display.max_columns',None)

## Credenciales

In [2]:
creds = json.load(open('credenciales_local.json','rb'))

## Crear conexión a base de datos

In [4]:
url = f"mysql+pymysql://{creds['user']}:{creds['password']}@{creds['servidor']}/ecobici"
cnx = create_engine(url,encoding='utf8')
cnx = cnx.connect()

In [5]:
cnx.closed

False

## Lectura y limpieza de datos

In [7]:
ruta = '/home/jose/Documentos/bd/ecobici/'

In [12]:
listaArchivos = sorted([os.path.join(ruta,arch) for arch in os.listdir(ruta) if '20' in arch])

In [13]:
len(listaArchivos)

62

In [14]:
datos = pd.read_csv(listaArchivos[0])

In [15]:
datos.shape

(810626, 9)

In [16]:
datos.head()

,Genero_Usuario,Edad_Usuario,Bici,Ciclo_Estacion_Retiro,Fecha_Retiro,Hora_Retiro,Ciclo_Estacion_Arribo,Fecha_Arribo,Hora_Arribo
0,M,42,1570,33,01/01/2017,0:05:06,1,01/01/2017,0:16:49
1,M,36,7083,27,01/01/2017,0:10:13,123,01/01/2017,0:18:28
2,M,18,4093,157,01/01/2017,0:14:06,157,01/01/2017,1:05:31
3,M,20,7704,87,01/01/2017,0:14:35,43,01/01/2017,0:18:27
4,M,34,7176,47,01/01/2017,0:16:54,123,01/01/2017,0:26:08


## Separación de Entidades

### Entidad Estación

In [36]:
entEstacion = pd.concat([datos[[f'Ciclo_Estacion_{tipo}']].rename(columns={f'Ciclo_Estacion_{tipo}':'idEstacion'}) for tipo in ['Arribo','Retiro']],ignore_index=True).drop_duplicates().reset_index(drop=True)
cat = pd.read_csv('/home/jose/Documentos/bd/ecobici/estaciones-de-ecobici.csv',usecols=['id','name','location_lat','location_lon','stationtype'])
cat.columns = ['idEstacion','nombre','latitud','longitud','tipoEstacion']
entEstacion = entEstacion.merge(cat,on='idEstacion',how='inner')

In [65]:
entEstacion.to_sql(con=cnx,
                   name='estacion',
                   if_exists='replace',
                   index=False,
                   chunksize=10000,
                   dtype={'idEstacion':INTEGER,
                          'nombre':VARCHAR(49),
                          'latitud':FLOAT,
                          'longitud':FLOAT,
                          'tipoEstacion':VARCHAR(13)})

447

### Entidad Bici

In [66]:
entBici = datos[['Bici']].drop_duplicates().reset_index(drop=True).rename(columns={'Bici':'idBici'})

In [67]:
entBici.to_sql(con=cnx,
                   name='bici',
                   if_exists='replace',
                   index=False,
                   chunksize=10000,
                   dtype={'idBici':INTEGER})

5123

In [68]:
datos.head()

,Genero_Usuario,Edad_Usuario,Bici,Ciclo_Estacion_Retiro,Fecha_Retiro,Hora_Retiro,Ciclo_Estacion_Arribo,Fecha_Arribo,Hora_Arribo
0,M,42,1570,33,01/01/2017,0:05:06,1,01/01/2017,0:16:49
1,M,36,7083,27,01/01/2017,0:10:13,123,01/01/2017,0:18:28
2,M,18,4093,157,01/01/2017,0:14:06,157,01/01/2017,1:05:31
3,M,20,7704,87,01/01/2017,0:14:35,43,01/01/2017,0:18:27
4,M,34,7176,47,01/01/2017,0:16:54,123,01/01/2017,0:26:08


### Entidad Viaje

In [69]:
entViaje = datos.copy()

In [70]:
for tipo in ['Arribo','Retiro']:
    entViaje[f'Hora_{tipo}'] = entViaje[f'Hora_{tipo}'].map(lambda x:f'0{x}' if len(x)==7 else x)
    entViaje[f'Fecha_{tipo}'] = entViaje[f'Fecha_{tipo}']+' '+entViaje[f'Hora_{tipo}']
    entViaje.drop(f'Hora_{tipo}',axis=1,inplace=True)
    entViaje[f'Fecha_{tipo}'] = pd.to_datetime(entViaje[f'Fecha_{tipo}'],
                                               format='%d/%m/%Y %H:%M:%S',
                                               errors='coerce')

In [71]:
entViaje.columns =['generoUsuario','edadUsuario','idBici','idEstacionRetiro','fechaRetiro','idEstacionArribo','fechaArribo']

In [72]:
entViaje.insert(0,'idViaje',entViaje.index+1)

In [74]:
entViaje.columns

Index(['idViaje', 'generoUsuario', 'edadUsuario', 'idBici', 'idEstacionRetiro',
       'fechaRetiro', 'idEstacionArribo', 'fechaArribo'],
      dtype='object')

In [76]:
entViaje = entViaje.merge(entEstacion,left_on='idEstacionRetiro',right_on='idEstacion',how='inner')
entViaje = entViaje.merge(entEstacion,left_on='idEstacionArribo',right_on='idEstacion',how='inner')
entViaje = entViaje.merge(entBici,left_on='idBici',right_on='idBici',how='inner')

In [81]:
entViaje.head()

,idViaje,generoUsuario,edadUsuario,idBici,idEstacionRetiro,fechaRetiro,idEstacionArribo,fechaArribo,idEstacion_x,nombre_x,latitud_x,longitud_x,tipoEstacion_x,idEstacion_y,nombre_y,latitud_y,longitud_y,tipoEstacion_y
0,1,M,42,1570,33,2017-01-01 00:05:06,1,2017-01-01 00:16:49,33,E33 LONDRES-FLORENCIA,19.424234,-99.166431,ELECTRIC_BIKE,1,1 RIO SENA-RIO BALSAS,19.433571,-99.167809,"BIKE,TPV"
1,771981,F,32,1570,136,2017-01-30 20:24:46,47,2017-01-30 20:28:21,136,136 ÁLVARO OBREGÓN-TONALÁ,19.417715,-99.162729,"BIKE,TPV",47,47 GLORIETA CIBELES-OAXACA,19.419515,-99.166299,"BIKE,TPV"
2,20581,M,46,1570,229,2017-01-02 18:38:14,21,2017-01-02 18:53:42,229,229 DICKENS-SENECA,19.430515,-99.203331,BIKE,21,21 REFORMA-DUBLIN,19.424720,-99.172060,"BIKE,TPV"
3,52169,M,23,1570,263,2017-01-04 07:04:06,130,2017-01-04 07:11:04,263,263 SADI CARNOT-GÓMEZ FARIAS,19.436274,-99.158425,"BIKE,TPV",130,130 MÉRIDA-DURANGO,19.421576,-99.158015,BIKE
4,352279,F,31,1570,158,2017-01-16 09:16:34,31,2017-01-16 09:29:00,158,158 HUATABAMPO-EJE 1PTE. CUAUHTÉMOC,19.407447,-99.155717,"BIKE,TPV",31,31 HAMBURGO-INSURGENTES,19.427838,-99.161193,BIKE


In [77]:
entViaje.shape

(810572, 18)

In [78]:
entViaje.to_sql(con=cnx,
                   name='viaje',
                   if_exists='replace',
                   index=False,
                   chunksize=10000,
                   dtype=dict(zip(entViaje.columns,
                                  [INTEGER,CHAR(1),INTEGER,INTEGER,INTEGER,DATETIME,INTEGER,DATETIME])))

810572

In [79]:
entViaje.shape

(810572, 18)

In [80]:
entViaje.head()

,idViaje,generoUsuario,edadUsuario,idBici,idEstacionRetiro,fechaRetiro,idEstacionArribo,fechaArribo,idEstacion_x,nombre_x,latitud_x,longitud_x,tipoEstacion_x,idEstacion_y,nombre_y,latitud_y,longitud_y,tipoEstacion_y
0,1,M,42,1570,33,2017-01-01 00:05:06,1,2017-01-01 00:16:49,33,E33 LONDRES-FLORENCIA,19.424234,-99.166431,ELECTRIC_BIKE,1,1 RIO SENA-RIO BALSAS,19.433571,-99.167809,"BIKE,TPV"
1,771981,F,32,1570,136,2017-01-30 20:24:46,47,2017-01-30 20:28:21,136,136 ÁLVARO OBREGÓN-TONALÁ,19.417715,-99.162729,"BIKE,TPV",47,47 GLORIETA CIBELES-OAXACA,19.419515,-99.166299,"BIKE,TPV"
2,20581,M,46,1570,229,2017-01-02 18:38:14,21,2017-01-02 18:53:42,229,229 DICKENS-SENECA,19.430515,-99.203331,BIKE,21,21 REFORMA-DUBLIN,19.424720,-99.172060,"BIKE,TPV"
3,52169,M,23,1570,263,2017-01-04 07:04:06,130,2017-01-04 07:11:04,263,263 SADI CARNOT-GÓMEZ FARIAS,19.436274,-99.158425,"BIKE,TPV",130,130 MÉRIDA-DURANGO,19.421576,-99.158015,BIKE
4,352279,F,31,1570,158,2017-01-16 09:16:34,31,2017-01-16 09:29:00,158,158 HUATABAMPO-EJE 1PTE. CUAUHTÉMOC,19.407447,-99.155717,"BIKE,TPV",31,31 HAMBURGO-INSURGENTES,19.427838,-99.161193,BIKE
